In [ ]:
pip install transformers datasets

In [3]:
import logging
import os
import torch
import transformers
from transformers import RobertaTokenizer, RobertaModel, RobertaConfig
import datasets
import json

In [34]:
max_source_length = 256
max_target_length = 30
truncate_variable_train = 5600
truncate_variable_dev = 700

In [ ]:
dataset ={'train':[], 'test':[], 'dev':[]}

In [ ]:
def convert_to_dataset(filename, stage, examples):
  idx = 0
  codefile = os.path.join(filename, stage + ".code")
  quesfile = os.path.join(filename, stage + ".question")
  ansfile = os.path.join(filename, stage + ".answer")
  with open(codefile,encoding="utf-8") as code_f:
      with open(ansfile, encoding="utf-8") as ans_f:
          with open(quesfile, encoding="utf-8") as ques_f:
              for codeline, quesline, ansline in zip(code_f,ques_f,ans_f):
                  code = codeline.strip()
                  question = quesline.strip()
                  ans = ansline.strip()
                  temp_list = {'code':code,'questions':question,'answer':ans}
                  examples[stage].append(temp_list)

In [ ]:
convert_to_dataset('/train', 'train', dataset)
convert_to_dataset('/valid', 'dev', dataset)
convert_to_dataset('/test', 'test', dataset)

FileNotFoundError: ignored

In [ ]:
json_object = json.dumps(dataset)
with open("codeQA.json", "w") as outfile:
    outfile.write(json_object)

In [6]:
dataset = datasets.DatasetDict()

In [ ]:
from datasets import DatasetDict, load_dataset
dataset = load_dataset('json', data_files='/codeQA.json', field='train')
dataset2 = load_dataset('json' ,data_files='/codeQA.json', field='test')
dataset3 = load_dataset('json',data_files='/codeQA.json', field='dev')

In [22]:
dataset['test'] = dataset2['train']
dataset['dev'] = dataset3['train']

In [14]:
def convert_examples_to_features(dataset, tokenizer, stage=None):
    features = []
    for example_index, example in enumerate(dataset[stage]):
        #source
        source_question_tokens = tokenizer.tokenize(example['questions'])
        source_code_tokens = tokenizer.tokenize(example['code'])
        source_tokens = source_question_tokens + [tokenizer.sep_token] + source_code_tokens
        source_tokens = source_tokens[:max_source_length-2]
        source_tokens =[tokenizer.cls_token]+source_tokens+[tokenizer.sep_token]
        source_ids =  tokenizer.convert_tokens_to_ids(source_tokens) 
        source_mask = [1] * (len(source_tokens))
        padding_length = max_source_length - len(source_ids)
        source_ids+=[tokenizer.pad_token_id]*padding_length
        source_mask+=[0]*padding_length
 
        #target
        if stage=="test":
            target_tokens = tokenizer.tokenize("None")
        else:
            target_tokens = tokenizer.tokenize(example['answer'])[:max_target_length-2]
        target_tokens = [tokenizer.cls_token]+target_tokens+[tokenizer.sep_token]            
        target_ids = tokenizer.convert_tokens_to_ids(target_tokens)
        target_mask = [1] *len(target_ids)
        padding_length = max_target_length - len(target_ids)
        target_ids+=[tokenizer.pad_token_id]*padding_length
        target_mask+=[0]*padding_length   

        features.append({'source_ids':source_ids ,'source_mask':source_mask ,'target_ids':target_ids ,'target_mask':target_mask})
    return features

In [ ]:
tokenizer = RobertaTokenizer.from_pretrained("microsoft/codebert-base")

In [15]:
dataset_token ={'train':[],'test':[],'dev':[]}

In [47]:
feature1 = convert_examples_to_features(dataset, tokenizer, stage='train')
feature2 = convert_examples_to_features(dataset, tokenizer, stage='test')
feature3 = convert_examples_to_features(dataset, tokenizer, stage='dev')


In [48]:
dataset_token['train'] = feature1
dataset_token['test'] = feature2
dataset_token['dev'] = feature3

In [49]:
dataset_token['train'] = dataset_token['train'][:truncate_variable_train]
dataset_token['test'] = dataset_token['test'][:truncate_variable_dev]
dataset_token['dev'] = dataset_token['dev'][:truncate_variable_dev]

In [50]:
len(dataset_token['test'][600]['source_mask'])

256

In [51]:
json_object = json.dumps(dataset_token, indent=1)
with open("token.json", "w") as outfile:
    outfile.write(json_object)